# Welcome to the EXR Converter Demo!

First, we will convert a sample FITS image to EXR format. To do this, we will use the fits2exr package. Import fits2exr (and exr2fits for later use) below.  

In [1]:
from exrconverter import fits2exr, exr2fits

Fits2exr contains one function, `convert()`, that performs all of the heavy lifting. The function header is copied here for convienience:

`def convert(input_fits, output_exr, output_pixel_type=None):`

The two required parameters are the name of the input FITS file and the name of the output EXR file. Optionally, an output pixel type can be included as well. Pixel options are: 

1. `FLOAT16`
2. `FLOAT32`
3. `FLOAT64`
4. `INT8`
5. `INT16`
6. `INT32`
7. `UINT16`

Because of the unique HDU formatting for FITS, only **2-D** images will be copied into the EXR file as different channels. The FITS header is much more sophisticated than that of EXR, so it is ported to EXR as a binary string. While this information is not available in EXR, it is converted back to a readable format when `exr2fits` is run. 

**TL/DR:** 
- A conversion from FITS to EXR and back to FITS will preserve the header and 2-D images from the original FITS file. 
- However, if `output_pixel_type` is smaller than the original fits type, the pixel data will be truncated. 

In [7]:
fits2exr.convert('frame-r-006793-3-0127.fits', 'exr_demo_output.exr')

An analogous function in exr2fits will convert EXR images to FITS format. The function header is: 

`def convert(input_exr, output_fits, output_pixel_type=None, verbose=True):`

The two required parameters, `input_exr` and `output_fits`, specify the input EXR and output FITS files, and `output_pixel_type` can be specified optionally. Additionally, the `verbose` parameter will print warnings.

In [8]:
exr2fits.convert('exr_demo_output.exr', 'fits_demo_output.fits')

## Converstion from FITS -> EXR -> FITS
We will compare `frame-r-006793-3-0127.fits` and `fits_demo_output.fits` to observe how the FITS file is modified by being stored in EXR.

First, we will check the number and types of HDUs.

In [2]:
from astropy.io import fits  

# Open the two fits files
hdu_list_original = fits.open('frame-r-006793-3-0127.fits')
hdu_list_modified = fits.open('fits_demo_output.fits')

print(f'The original fits file has {len(hdu_list_original)} HDU(s).')
print(f'The original fits file has {len(hdu_list_modified)} HDU(s).')

The original fits file has 4 HDU(s).
The original fits file has 1 HDU(s).


It is clear that only **one** HDU from the original FITS file survives the storage in EXR. We will check if this was the only 2-D image in the original FITS file. 

In [25]:
for hdu in hdu_list_original:
    if hdu.is_image and len(hdu.data.shape) == 2:
        print(f"The {hdu.name} HDU is a 2-D image.")

hdu_list_original.close()
hdu_list_modified.close()

The PRIMARY HDU is a 2-D image.


Correct!

We will now observe the truncation of FITS data if a smaller type is used to store in EXR.

In [11]:
import numpy as np

# Run fits2exr with 16-bit float type
fits2exr.convert('frame-r-006793-3-0127.fits', 'exr_demo_output.exr', np.float16)
exr2fits.convert('exr_demo_output.exr', 'fits_demo_output.fits', np.float32)

# Open the two fits files again
hdu_list_original = fits.open('frame-r-006793-3-0127.fits')
hdu_list_modified = fits.open('fits_demo_output.fits')

image_original = hdu_list_original[0].data
image_modified = hdu_list_modified[0].data

print (f"The difference between pixels is {np.float32(image_original[0][0]) - np.float32(image_modified[0][0])}.")

The difference between pixels is 0.0.


In [6]:
image_original[0][0].dtype

dtype('float32')

In [9]:
import OpenEXR


exr_file = OpenEXR.InputFile('exr_demo_output.exr')

In [10]:
exr_header = exr_file.header()
print (exr_header)

{'channels': {'0': HALF (1, 1)}, 'compression': ZIP_COMPRESSION, 'dataWindow': (0, 0) - (2047, 1488), 'displayWindow': (0, 0) - (2047, 1488), 'fits_headers': b'["SIMPLE  =                    T /                                                BITPIX  =                  -32 / 32 bit floating point                          NAXIS   =                    2                                                  NAXIS1  =                 2048                                                  NAXIS2  =                 1489                                                  EXTEND  =                    T /Extensions may be present                       BZERO   =              0.00000 /Set by MRD_SCALE                                BSCALE  =              1.00000 /Set by MRD_SCALE                                TAI     =        4683768020.68 / 1st row Number of seconds since Nov 17 1858    RA      =            179.05560 / 1st row RA of telescope boresight (deg)        DEC     =            0.000000  / 1st r